Create KeyPair

In [ ]:
import boto3

In [ ]:
ec2 = boto3.client('ec2')
ec2.describe_instances()

In [ ]:
resp = ec2.create_key_pair(KeyName="kev-ml")

In [ ]:
file = open("creds/kev-ml.pem", "w")
file.write(resp["KeyMaterial"])
file.close()

Programmatically Create EC2 Instance

In [ ]:
# Info found in EC2 launch page
response = ec2.run_instances(
    ImageId = "ami-01816d07b1128cd2d",
    Name = "kev-ml",
    MinCount = 1,
    MaxCount = 1,
    InstanceType = "t2.micro",
    KeyName = "kev-ml",
    BlockDeviceMappings = [
        {
            "DeviceName": "/dev/xvda",
            "Ebs": {
                "DeleteOnTermination": True,
                "VolumeSize": 20
            }
        }
    ]
)

In [ ]:
response

Programmatically Create Security Groups

In [ ]:
response = ec2.describe_security_groups()

response = ec2.create_security_group(
    GroupName = "kev-ml",
    Description = "Security group for testing"
)

In [ ]:
security_group_id = response["GroupId"]
security_group_id

In [ ]:
# ip, port, traffic type
response = ec2.authorize_security_group_ingress(
    GroupId = security_group_id,
    IpPermissions = [
        {
            "IpProtocol": "tcp",
            "FromPort": 22,
            "ToPort": 22,
            "IpRanges": [{"CidrIp": "0.0.0.0/0"}]
        },
        {
            "IpProtocol": "tcp",
            "FromPort": 80,
            "ToPort": 80,
            "IpRanges": [{"CidrIp": "0.0.0.0/0"}]
        }
    ]
)

In [ ]:
response = ec2.describe_instances()
instance_id = response["Reservations"][0]["Instances"][0]["InstanceId"]
instance_id, security_group_id

In [ ]:
sg_gid = response["Reservations"][0]["Instances"][0]["SecurityGroups"][0]["GroupId"]


In [ ]:
ec2.modify_instance_attribute(InstanceId=instance_id, Groups=[security_group_id])

Programmatically Start, Stop, and Delete EC2 Instances

In [ ]:
response = ec2.describe_instances(InstanceIds=[instance_id])
response

In [ ]:
# status -> running, stopped, termianted, pending, etc.

import time

def wait_for_status(instance_id, target_status):
    while True:
        response = ec2.describe_instances(InstanceIds=[instance_id])
        status = response["Reservations"][0]["Instances"][0]["State"]["Name"]

        if status == target_status:
            print(f"status is in {target_status} state")
            break

        time.sleep(10)


In [ ]:
def start_instances(instance_id):
    ec2.start_instances(InstanceIds=[instance_id])
    wait_for_status(instance_id, "running")

start_instances(instance_id)

In [ ]:
def stop_instances(instance_id):
    ec2.stop_instances(InstanceIds=[instance_id])
    wait_for_status(instance_id, "stopped")

stop_instances(instance_id)

In [ ]:
def terminate_instances(instance_id):
    ec2.terminate_instances(InstanceIds=[instance_id])
    wait_for_status(instance_id, "terminated")

terminate_instances(instance_id)